In [354]:
import pandas as pd
import datetime

In [393]:
participant_data = pd.read_excel("data/data_collection.xslx", sheet_name = "40ab", skiprows = 3)
participant_data.drop(["Unnamed: 4", "Unnamed: 5", "Unnamed: 6", "Summary Questions"], axis = 1, inplace = True)
participant_data = participant_data[['Timestamp', 'Theme']]

In [394]:
participant_data = participant_data[participant_data["Timestamp"].notna()]
participant_data = participant_data[~participant_data["Timestamp"].str.contains("\*")]
participant_data.reset_index(inplace=True)

In [395]:
nba_idx = participant_data[participant_data["Timestamp"] == "NBA NOTEBOOK"].index.values.astype(int)[0]
nba_idx

70

In [396]:
participant_data[['event_start', 'event_end']] = participant_data.Timestamp.str.split('-', expand = True)
participant_data.drop('Timestamp', axis = 1, inplace=True)
participant_data.event_start = participant_data.event_start.str.strip().copy(deep=True)
participant_data.event_start = participant_data.event_start.str.replace('"', '').copy(deep=True)
participant_data.event_start = participant_data.event_start.str.replace("'", '').copy(deep=True)
participant_data.event_end = participant_data.event_end.str.strip().copy(deep=True)
participant_data.event_end = participant_data.event_end.str.replace("'", '').copy(deep=True)
participant_data.event_end = participant_data.event_end.str.replace('"', '').copy(deep=True)

In [397]:
elections_notebook = participant_data.iloc[:nba_idx, :]
nba_notebook = participant_data.iloc[nba_idx + 1:, :]

In [398]:
# From this person because I was lazy: https://stackoverflow.com/a/10748024
def time_in_range(start, end, x):
    """Return true if x is in the range [start, end]"""
    if start <= end:
        return start <= x <= end
    else:
        return start <= x or x <= end

In [402]:
def parent_finder(df_values, verbose = False):
    results = []
    
    for idx, value in enumerate(df_values):
        scanned_entries = []

        if value[3] is None:
            continue
        if idx == len(df_values):
            break
        start = datetime.datetime.strptime(value[2], "%M:%S").time()
        end = datetime.datetime.strptime(value[3], "%M:%S").time()
        for events in df_values:
            if events[3] is None:
                if events[0] in scanned_entries:
                    continue
                event_of_interest = datetime.datetime.strptime(events[2], "%M:%S").time()
                scanned_entries.append(events[0])

                if time_in_range(start, end, event_of_interest):
                    result = (value[0], events[0])
                    results.append(result)
                    if verbose:
                        print(result)
    return results

In [403]:
parent_finder(elections_notebook.values)

[(0, 1),
 (0, 2),
 (13, 15),
 (14, 15),
 (16, 15),
 (17, 18),
 (38, 37),
 (43, 45),
 (46, 48),
 (47, 48),
 (52, 53)]

In [404]:
parent_finder(nba_notebook.values)

[(82, 83),
 (82, 84),
 (82, 85),
 (82, 86),
 (82, 88),
 (82, 89),
 (82, 90),
 (82, 91),
 (82, 92),
 (82, 93),
 (82, 94),
 (82, 95),
 (117, 118)]